# Actividad: Problemas de búsqueda

- Diseño de agentes inteligentes
- Profesor:
- Equipo:

| Nombre | Matrícula |
| ----- | ---- |
| Juan Pablo Echeagaray González | A0083646 |
| Emily Rebeca Méndez Cruz | A00830768 |
| Oscar Antonio Banderas Álvarez | A01568492 |

In [93]:
import numpy as np
from simpleai.search import SearchProblem
from simpleai.search import astar, breadth_first, uniform_cost

## Problema 3

## Problema 4: Rompecabezas deslizante

1. Utilicen el algoritmo A* para resolver el rompecabezas deslizante de 8 números (3x3). Utilicen como función heurística la suma de las distancias entre la posición de cada número y su correspondiente posición objetivo. Corran su algoritmo con diferentes estados iniciales.
2. Cambien la función heurística por la cantidad elementos que están en la posición incorrecta. ¿Cuál función heurística consideran que es la más apropiada?
3. Utilicen un algoritmo de búsqueda no informada  para solucionar el rompecabezas de 3x3. ¿El algoritmo fue capaz de resolver el problema en tiempo razonable? ¿Qué algoritmo consideras que es más eficiente, el de búsqueda informada o el de búsqueda no informada?
4. Utilicen el algoritmo A* con la función heurística de su preferencia para resolver el rompecabezas deslizante de 15 números (4x4). ¿El algoritmo seleccionado es capaz de encontrar una solución en estos casos?


Implementación basada fuertemente en [este repositorio](https://github.com/simpleai-team/simpleai/blob/master/samples/search/eight_puzzle.py)

In [123]:
GOAL = '''1-2-3
4-5-6
7-8-E'''

INITIAL = '''4-1-2
7-E-3
8-5-6'''

INITIAL_2 = '''2-3-7
1-4-6
8-5-E'''

INITIAL_3 = '''7-E-4
1-8-6
2-3-5'''

INITIAL_4 = '''5-2-8
4-6-1
3-E-7'''

initial_states = [INITIAL, INITIAL_2, INITIAL_3, INITIAL_4]

In [124]:
def list_to_string(list_):
    return '\n'.join(['-'.join(row) for row in list_])


def string_to_list(string_):
    return [row.split('-') for row in string_.split('\n')]


def piece_finder(rows, element_to_find):
    for ir, row in enumerate(rows):
        for ic, element in enumerate(row):
            if element == element_to_find:
                return ir, ic


In [131]:
goal_positions = {}
rows_goal = string_to_list(GOAL)
for number in '12345678E':
    goal_positions[number] = piece_finder(rows_goal, number)


In [126]:
class sliding_puzzle(SearchProblem):

    def __init__(self, initial_state: str, goal_state: str):
        self.initial = initial_state
        self.goal = goal_state

        SearchProblem.__init__(self, initial_state)

    def actions(self, state):
        rows = string_to_list(state)
        row_e, col_e = piece_finder(rows, 'E')

        actions = []
        if row_e > 0:
            actions.append(rows[row_e - 1][col_e])
        if row_e < 2:
            actions.append(rows[row_e + 1][col_e])
        if col_e > 0:
            actions.append(rows[row_e][col_e - 1])
        if col_e < 2:
            actions.append(rows[row_e][col_e + 1])

        return actions


    def result(self, state, action):
        rows = string_to_list(state)
        row_e, col_e = piece_finder(rows, 'E')
        row_n, col_n = piece_finder(rows, action)

        rows[row_e][col_e], rows[row_n][col_n] = rows[row_n][col_n], rows[row_e][col_e]

        return list_to_string(rows)


    def cost(self, state1, action, state2):
        return 1


    def is_goal(self, state):
        return state == self.goal


    def heuristic(self, state):
        rows = string_to_list(state)

        distance = 0

        for number in '12345678E':
            row_n, col_n = piece_finder(rows, number)
            row_n_goal, col_n_goal = goal_positions[number]

            distance += abs(row_n - row_n_goal) + abs(col_n - col_n_goal)

        return distance
        

#### Soluciones iniciales

In [127]:
%%timeit
result = astar(sliding_puzzle(INITIAL, GOAL), graph_search=True)

631 µs ± 61.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [128]:
print(f'Resuelto en {result.cost} movimientos')
for action, state in result.path():
    print(f'Move number {action}')
    print(state)

Resuelto en 8 movimientos
Move number None
4-1-2
7-E-3
8-5-6
Move number 5
4-1-2
7-5-3
8-E-6
Move number 8
4-1-2
7-5-3
E-8-6
Move number 7
4-1-2
E-5-3
7-8-6
Move number 4
E-1-2
4-5-3
7-8-6
Move number 1
1-E-2
4-5-3
7-8-6
Move number 2
1-2-E
4-5-3
7-8-6
Move number 3
1-2-3
4-5-E
7-8-6
Move number 6
1-2-3
4-5-6
7-8-E


In [129]:
def tester(initial_state, goal_state):
    result = astar(sliding_puzzle(initial_state, goal_state), graph_search=True)
    print(f'Resuelto en {result.cost} movimientos')
    for action, state in result.path():
        print(f'Move number {action}')
        print(state)

In [130]:
for initial_state in initial_states:
    tester(initial_state, GOAL)

Resuelto en 8 movimientos
Move number None
4-1-2
7-E-3
8-5-6
Move number 5
4-1-2
7-5-3
8-E-6
Move number 8
4-1-2
7-5-3
E-8-6
Move number 7
4-1-2
E-5-3
7-8-6
Move number 4
E-1-2
4-5-3
7-8-6
Move number 1
1-E-2
4-5-3
7-8-6
Move number 2
1-2-E
4-5-3
7-8-6
Move number 3
1-2-3
4-5-E
7-8-6
Move number 6
1-2-3
4-5-6
7-8-E
Resuelto en 18 movimientos
Move number None
2-3-7
1-4-6
8-5-E
Move number 6
2-3-7
1-4-E
8-5-6
Move number 7
2-3-E
1-4-7
8-5-6
Move number 3
2-E-3
1-4-7
8-5-6
Move number 2
E-2-3
1-4-7
8-5-6
Move number 1
1-2-3
E-4-7
8-5-6
Move number 4
1-2-3
4-E-7
8-5-6
Move number 5
1-2-3
4-5-7
8-E-6
Move number 8
1-2-3
4-5-7
E-8-6
Move number 4
1-2-3
E-5-7
4-8-6
Move number 5
1-2-3
5-E-7
4-8-6
Move number 7
1-2-3
5-7-E
4-8-6
Move number 6
1-2-3
5-7-6
4-8-E
Move number 8
1-2-3
5-7-6
4-E-8
Move number 7
1-2-3
5-E-6
4-7-8
Move number 5
1-2-3
E-5-6
4-7-8
Move number 4
1-2-3
4-5-6
E-7-8
Move number 7
1-2-3
4-5-6
7-E-8
Move number 8
1-2-3
4-5-6
7-8-E
Resuelto en 21 movimientos
Move number None
7

#### Cambio de heurística

Para el caso de nuestra implementación no aplicaría realizar ese cambio de heurística, ya que esa es la que el algoritmo usa desde un comienzo

#### Algoritmo de búsqueda no informada

In [120]:
%%timeit
result_uninformed = uniform_cost(sliding_puzzle(INITIAL, GOAL), graph_search=True)


17.2 ms ± 3.16 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [115]:
print(f'Resuelto en {result.cost} movimientos')
for action, state in result.path():
    print(f'Move number {action}')
    print(state)
    

Resuelto en 8 movimientos
Move number None
4-1-2
7-E-3
8-5-6
Move number 5
4-1-2
7-5-3
8-E-6
Move number 8
4-1-2
7-5-3
E-8-6
Move number 7
4-1-2
E-5-3
7-8-6
Move number 4
E-1-2
4-5-3
7-8-6
Move number 1
1-E-2
4-5-3
7-8-6
Move number 2
1-2-E
4-5-3
7-8-6
Move number 3
1-2-3
4-5-E
7-8-6
Move number 6
1-2-3
4-5-6
7-8-E


Al menos el algoritmo de búsqueda uniforme (haciendo uso de una búsqueda de grafos) fue capaz de resolver el problema 3x3 en un tiempo aceptable y con el mismo número de movimientos. Sin embargo, gracias a la función mágica `%%timeit` podemos ver como el algoritmo de búsqueda no informada no representa competencia alguna para `A*`. El tiempo promedio de ejecución de este último es al menos 2 ordenes de magnitud menos que el de búsqueda uniforme. (Al tiempo de creación de este documento, `A*` fue 27 veces más rápido).

#### Problema deslizante 4x4

In [132]:
GOAL_4BY4 = '''1-2-3-4
5-6-7-8
9-10-11-12
13-14-15-E'''

INITIAL_4BY4 = '''13-3-5-9
12-4-1-7
6-8-2-E
15-10-14-11'''

In [133]:
sliding_4by4 = sliding_puzzle(INITIAL_4BY4, GOAL_4BY4)

In [135]:
# %%timeit
result_4by4 = astar(sliding_4by4, graph_search=True)

KeyboardInterrupt: 

Todo parece indicar que el algoritmo no es capaz de resolverlo en un tiempo óptimo, dado que `A*` es completo, sabemos que en algún momento encontrará una solución, pero para este caso, le tomará mucho tiempo encontrarla.

## Problema 5: Laberintos

Utilicen el algoritmo A* para encontrar un camino entre los dos puntos indicados en el siguiente laberinto.

```txt
++++++++++++++++++++++
+ O +   ++ ++        +
+     +     +++++++ ++
+ +    ++  ++++ +++ ++
+ +   + + ++         +
+          ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + +  + +  +
+++++ +  + + +     X +
++++++++++++++++++++++
```

- El símbolo + indica un obstáculo o pared. El símbolo O representa el punto de inicio, y X indica el objetivo.
- Inventen otros tres laberintos, y prueben de nuevo el algoritmo A*.

> Nota: Usen como heurística la distancia entre el punto actual y el punto objetivo.

Las implementaciones para la solución de este problema tomaron una gran inspiración de [este repositorio](https://github.com/simpleai-team/simpleai/blob/master/samples/search/game_walk.py)

In [ ]:
maze = '''
++++++++++++++++++++++
+ O +   ++ ++        +
+     +     +++++++ ++
+ +    ++  ++++ +++ ++
+ +   + + ++         +
+          ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + +  + +  +
+++++ +  + + +     X +
++++++++++++++++++++++
'''

maze_2 = '''
++++++++++++++++++++++
+   +   ++O++        +
+     +     +++++++ ++
+ +    ++  ++++ +++ ++
+ +   + + ++         +
+          ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+X         + +  + +  +
+++++ +  + + +       +
++++++++++++++++++++++
'''

maze_3 = '''
++++++++++++++++++++++
+ O +   ++ ++        +
+     +     +++++++ ++
+ +        ++++ +++ ++
+ +   + +            +
+          ++  ++  + +
+++++++ +  ++  ++  + +
+++++ ++++++ +  ++   +
+          + +  + +  +
+++++ + X            +
++++++++++++++++++++++
'''

maze_4 = '''
++++++++++++++++++++++
+ O +   ++ ++        +
++    +     +++++++ ++
+++    ++    ++ +++ ++
+++++++ + ++         +
+X         ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + +  + +  +
+++++ +  + + +       +
++++++++++++++++++++++
'''


In [ ]:
COSTS = {
    'up': 1,
    'down': 1,
    'left': 1,
    'right': 1,
}

In [ ]:
class Labyrinth(SearchProblem):
    def __init__(self, maze):
        """Constructor for the labyrinth game.

        Args:
            - maze (list[list]): Array representation of the labyrinth
        """        
        self.board = maze

        for y in range(len(self.board)):
            for x in range(len(self.board[y])):
                if self.board[y][x].lower() == 'x':
                    self.goal = (x, y)
                elif self.board[y][x].lower() == 'o':
                    self.start = (x, y)
        super(Labyrinth, self).__init__(initial_state=self.start)

    
    def actions(self, state):
        """Returns a list of all possible actions from the current state.

        Args:
            - state (tuple(int)): x, y coordinates of the current position

        Returns:
            - list[str]: list of all possible actions from the current state
        """        
        actions = []

        for action in list(COSTS.keys()):
            x, y = self.result(state, action)
            if self.board[y][x] != '+':
                actions.append(action)

        return actions

    
    def result(self, state, action):
        """Result function for the labyrinth game.

        Args:
            - state (tuple(int)): x, y coordinates of the current position
            - action (str): Movement to be made

        Returns:
            - tuple(int): New position coordinates
        """        
        x, y = state
        if action.count('up'):
            y -= 1
        if action.count('down'):
            y += 1
        if action.count('left'):
            x -= 1
        if action.count('right'):
            x += 1

        new_state = (x, y)
        return new_state


    def is_goal(self, state):
        """Check if the current state is the goal state.

        Args:
            - state (tuple(int)): x, y coordinates of the current position

        Returns:
            - bool: whether the current state is the goal state
        """        
        return state == self.goal


    def cost(self, state, action, state2):
        """Returns cost of the movement.

        Args:
            - state (tuple(int)): x, y coordinates of the current position
            - action (str): movement to be made
            - state2 (tuple(int)): x, y coordinates of the new position

        Returns:
            - float: cost of the movement
        """        
        return COSTS[action]


    def heuristic(self, state):
        """Heuristic function for the labyrinth game.

        Args:
            - state (tuple(int)): x, y coordinates of the current position

        Returns:
            - float: Euclidean distance to the goal state
        """        
        x, y = state
        xg, yg = self.goal
        return np.sqrt((x - xg)**2 + (y - yg)**2)
    

In [ ]:
def solver(maze):
    """Maze solver.

    Args:
        - maze (list[list]): Array representation of the labyrinth
    """    
    MAZE = [list(x) for x in maze.split('\n') if x]
    problem = Labyrinth(MAZE)
    result = astar(problem, graph_search=True)
    
    path = [x[1] for x in result.path()]

    for y in range(len(MAZE)):
        for x in range(len(MAZE[y])):
            if (x, y) == problem.start:
                print("o", end='')
            elif (x, y) == problem.goal:
                print("x", end='')
            elif (x, y) in path:
                print(".", end='')
            else:
                print(MAZE[y][x], end='')
        print()


In [ ]:
print('Laberinto 1')
solver(maze)

Laberinto 1
++++++++++++++++++++++
+ o.+   ++ ++        +
+  ...+     +++++++ ++
+ +  . ++  ++++ +++ ++
+ +  .+ + ++         +
+    ......++  ++  + +
+++++ + + .....++  + +
+++++ +++  + +..++   +
+          + + .+ +  +
+++++ +  + + + ....x +
++++++++++++++++++++++


In [ ]:
print('Laberinto 2')
solver(maze_2)

Laberinto 2
++++++++++++++++++++++
+   +   ++o++        +
+     +  .. +++++++ ++
+ +    ++. ++++ +++ ++
+ +   + +.++         +
+    ..... ++  ++  + +
+++++.+ +      ++  + +
+++++.+++  + +  ++   +
+x....     + +  + +  +
+++++ +  + + +       +
++++++++++++++++++++++


In [ ]:
solver(maze_3)

++++++++++++++++++++++
+ o +   ++ ++        +
+ ..  +     +++++++ ++
+ +....... ++++ +++ ++
+ +   + +.....       +
+          ++. ++  + +
+++++++ +  ++..++  + +
+++++ ++++++ +. ++   +
+          + +. + +  +
+++++ + x......      +
++++++++++++++++++++++


In [ ]:
solver(maze_4)

++++++++++++++++++++++
+ o +...++ ++        +
++....+...  +++++++ ++
+++    ++.   ++ +++ ++
+++++++ +.++         +
+x........ ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + +  + +  +
+++++ +  + + +       +
++++++++++++++++++++++
